# **<p style="text-align: center;">Исследование надежности заемщиков.</p>**

## Загрузка и подготовка данных

### Импортируем библиотеку pandas. Считываем данные из csv-файла в датафрейм и сохраняем в переменную `data`. 

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

### Выводим первые 20 строчек датафрейма `data` на экран.

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Выводим основную информацию о датафрейме с помощью метода `info()`.

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Выведим количество пропущенных значений для каждого столбца. 

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них  `days_employed`. Пропуски в этом столбце обработаем на следующем этапе. Другой столбец с пропущенными значениями  `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполним пропуски в этом столбце медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` будет заполнен медианным доходом среди всех записей с тем же типом.

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработаем значения в этом столбце: заменим все отрицательные значения положительными с помощью метода `abs()`.

In [6]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведим медианное значение трудового стажа `days_employed` в днях.

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получаются аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть. Тем более этот столбец не понадобится для исследования.

Выведим перечень уникальных значений столбца `children`.

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Ещё раз выведим перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

Заполним пропуски в столбце `days_employed` медианными значениями для каждого типа занятости `income_type`.

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Убедимся что все пропуски заполнены. Выведим количество пропущенных значений для каждого столбца.

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

### Заменим вещественный тип данных в столбце `total_income` на целочисленный .

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру. 

In [14]:
data['education'] = data['education'].str.lower()

Выведим на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их.

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`.**

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Выведим на экран перечень уникальных целей взятия кредита из столбца `purpose`.

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Исследуем данные и ответим на вопросы.

#### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [22]:
data.groupby('children')['debt'].agg('sum')

children
0    1063
1     444
2     194
3      27
4       4
5       0
Name: debt, dtype: int64

In [23]:
relationship_children = data.pivot_table(index=['children'], values='debt', aggfunc= {'sum', 'count'})
relationship_children = relationship_children.rename(columns={"sum": "debtors",'count': 'all_customers'})

In [24]:
relationship_children

,all_customers,debtors
children,,
0,14091,1063
1,4808,444
2,2052,194
3,330,27
4,41,4
5,9,0


In [25]:
relationship_children['no_debtors'] = relationship_children['all_customers'] - relationship_children['debtors']
relationship_children['all_debtors'] = round(relationship_children['debtors']/ relationship_children['all_customers'],3)
relationship_children['all_no_debtors'] = round(relationship_children['no_debtors']/ relationship_children['all_customers'],3)
relationship_children

,all_customers,debtors,no_debtors,all_debtors,all_no_debtors
children,,,,,
0,14091,1063,13028,0.075,0.925
1,4808,444,4364,0.092,0.908
2,2052,194,1858,0.095,0.905
3,330,27,303,0.082,0.918
4,41,4,37,0.098,0.902
5,9,0,9,0.000,1.000


Для каждой категории посчитано количество должников, количество людей, которые вернули кредит в срок, и их долю от всех клиентов банка. 
    
Стоит учитывать, что данных по семьям с 3, 4, 5 детьми очень мало, и их не стоит брать в расчет. 
    
**Вывод:** клиенты,у которых есть дети имеют большую долю невозврата кредита в срок,в свою очередь, клиенты, у которых нет детей имеют меньшую долю быть должником перед банком.


#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [26]:
relationship_family_status= data.pivot_table(index=['family_status'], values='debt', aggfunc= {'sum', 'count', 'mean'})
pd.set_option('display.float_format', '{:.3f}'.format)
relationship_family_status['count'] = relationship_family_status['count'].astype (int)
relationship_family_status['sum'] = relationship_family_status['sum'].astype (int)
relationship_family_status = relationship_family_status.rename(columns={"sum": "debtors",'count': 'all_customers','mean': 'share_of_debtors' })
relationship_family_status

,all_customers,share_of_debtors,debtors
family_status,,,
Не женат / не замужем,2796,0.098,273
в разводе,1189,0.071,84
вдовец / вдова,951,0.066,63
гражданский брак,4134,0.093,385
женат / замужем,12261,0.076,927


**Вывод:**  Заемщики, состоящие или состоявшие в браке реже допускают просрочку. Самая большая доля должников у клиентов, у которых статус семейного положения не женат / не замужем,либо гражданский брак,самая маленькая доля должников среди клиентов,у которых семейное положение вдовец / вдова.

#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [27]:
relationship_total_income = data.pivot_table(index=['total_income_category'], values='debt', aggfunc= {'sum', 'count', 'mean'})
pd.set_option('display.float_format', '{:.3f}'.format)
relationship_total_income['count'] = relationship_total_income['count'].astype (int)
relationship_total_income['sum'] = relationship_total_income['sum'].astype (int)
relationship_total_income = relationship_total_income.rename(columns={"sum": "debtors",'count': 'all_customers','mean': 'share_of_debtors' }) 
relationship_total_income

,all_customers,share_of_debtors,debtors
total_income_category,,,
A,25,0.080,2
B,5014,0.071,354
C,15921,0.085,1353
D,349,0.060,21
E,22,0.091,2


**Вывод:** Заемщики с более высоким уровнем дохода реже допускают просрочку. Самая большая доля должников у клиентов,который относятся к категории Е,в совю очередь самая маленькая доля должников,которые относятся к категории D. 

#### Как разные цели кредита влияют на его возврат в срок?

In [28]:
relationship_purpose= data.pivot_table(index=['purpose_category'], values='debt', aggfunc= {'sum', 'count', 'mean'})
pd.set_option('display.float_format', '{:.3f}'.format)
relationship_purpose['count'] = relationship_purpose['count'].astype (int)
relationship_purpose['sum'] = relationship_purpose['sum'].astype (int)
relationship_purpose = relationship_purpose.rename(columns={"sum": "debtors",'count': 'all_customers','mean': 'share_of_debtors' }) 
relationship_purpose

,all_customers,share_of_debtors,debtors
purpose_category,,,
операции с автомобилем,4279,0.093,400
операции с недвижимостью,10751,0.073,780
получение образования,3988,0.093,369
проведение свадьбы,2313,0.079,183


**Вывод:** По таблице видно,что самая большая доля должников сосредоточенна у клиентов целью кредита которых является операции с автомобилем, либо получение образования,меньшая доля должников у клиентов,цель кредита которых операции с недвижимостью.

#### Приведите возможные причины появления пропусков в исходных данных.

*Ответ:*  В двух столбцах были пропущенные значения. Один из них — `days_employed` - хранит данные об общем трудовом стаже в днях . Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. Колличество пропусков в столбцах одинаковое, что свидетельствует о специфике заполнения анкеты при получении кредита. Данные поля анкеты при получении кредита не являются обязательными, записи в них отсутствуют.

####  Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ:* Заполнить пропуски медианным значением — лучшее решение для количественных переменных,так как медианное значение лучше всего отражает выборку, чем среднее арифметическое, поскольку устойчива к искажениям. Медиана в отличие от среднего значения, менее чувствительна к выбросам, среднее значение смещается в сторону выбросов.

### Общий вывод.

На этапе обработки аномальных значений,помимо обработанных артефактов выявленно аномально большое значение в столбце `days_employed` , которое хранит данные об общем трудовом стаже в днях. У клиента колличество дней трудового стажа анамально высокое,что привело к неккоректному  оотображению статистических данных, которые дают представление о распределении значений.

In [29]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21331.000,21331.000,21331.000,21331.000,21331.000,21331.000,21331.000
mean,0.474,67148.972,43.278,0.817,0.974,0.081,165342.746
std,0.752,139281.142,12.575,0.549,1.422,0.273,98313.986
min,0.000,24.142,0.000,0.000,0.000,0.000,20667.000
25%,0.000,1024.525,33.000,1.000,0.000,0.000,107507.000
50%,0.000,1996.733,42.000,1.000,0.000,0.000,142594.000
75%,1.000,5337.760,53.000,1.000,1.000,0.000,195842.000
max,5.000,401755.400,75.000,4.000,4.000,1.000,2265604.000


In [30]:
anomaly_days_employed = data.loc[data['days_employed'] >= 401755]
anomaly_days_employed

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
6954,0,401755.400,56,среднее,1,вдовец / вдова,2,F,пенсионер,0,176278,ремонт жилью,C,операции с недвижимостью


In [31]:
anomaly_year_employed = anomaly_days_employed['days_employed']/365
anomaly_year_employed

6954   1100.700
Name: days_employed, dtype: float64

***Не все гипотезы подтвердились. Прослеживается зависимость между семейным положением, уровнем дохода,цели кредита и возвратом кредита в срок. 
Исходя из данных выше можно описать клиента с наибольшей вероятностью просрочки следующим образом***: 
- семейное положением: вдовец / вдова
- уровень дохода: категория D (30001–50000)
- цель кредита: операции с недвижимостью


**<p style="text-align: center;">Итоговый вывод.</p>**

1. Цель работы - исследование надежности заемщиков банка.В ходе работы над проектом были реализованны следующие шаги:
    - удаление пропусков (основные причины их появления — технические ошибки и человеческий фактор)
    - замена на медианное значение (так как медианное значение лучше всего отражает выборку, чем среднее арифметическое, поскольку устойчива к искажениям)
    - обработка аномальных значений
    - обработка дубликатов
    
    
    
2. Исследуя данные были полученны ответы на вопросы:

   - Есть ли зависимость между количеством детей и возвратом кредита в срок? 
    Да, клиенты,у которых есть дети имеют большую долю невозврата кредита в срок,в свою очередь, клиенты, у которых нет детей имеют меньшую долю быть должником перед банком.

   - Есть ли зависимость между семейным положением и возвратом кредита в срок?
    Да,самая большая доля должников у клиентов, у которых статус семейного положения не женат / не замужем,либо гражданский брак,самая маленькая доля должников среди клиентов,у которых семейное положение вдовец / вдова.

   - Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
    Да,самая большая доля должников у клиентов,которые относятся к категории Е,в свою очередь самая маленькая доля должников,которые относятся к категории D. 
    
   - Как разные цели кредита влияют на его возврат в срок?
    Самая большая доля должников сосредоточенна у клиентов целью кредита которых является операции с автомобилем либо получение образования,меньшая доля должников у клиентов,цель кредита которых операции с недвижимостью.
    

